In [32]:
import pathlib

from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import openai

EVENTBRITE_URL = 'https://www.eventbrite.com/d/ca--san-francisco/free--business--events--this-week/'
links = []


In [34]:
response = openai.Completion.create(
        model="text-davinci-003",
        prompt='What is the date exactly 15 from today? Provide a text format such as "Feb 22".',
        temperature=0.0,
        max_tokens = 50,
  )
stop_date = response['choices'][0]['text'].strip()
print(stop_date)

March 9


In [35]:
sf_events = pd.DataFrame()
done = False
for val in range(1, 20):

    try:
        page = requests.get(f'{EVENTBRITE_URL}/?page={val}')                                 
        soup = BeautifulSoup(page.content, 'html.parser')
    except:
        break

    event_elems = soup.find_all('div', class_='eds-event-card-content__primary-content')
    sec_event_elems = soup.find_all('div', class_='eds-event-card-content__sub-content')

    if done == True:
        break


    for event,sec_event in zip(event_elems,sec_event_elems):
        eventbrite = {}
        link = event.find('a',href=True)['href']

        if link not in links:
            links.append(link)
            try:
                data = sec_event.find_all('div',class_='eds-event-card-content__sub eds-text-bm eds-text-color--ui-600 eds-l-mar-top-1')
                venue = ''
                for item in data:
                    venue += item.text
            except:
                venue = ''
            response = requests.get(link)
            soup = BeautifulSoup(response.content, 'html.parser')
            event_name = soup.find('h1', class_='event-title').get_text()
            event_date = soup.find('p', class_='date-and-time').get_text()
            if stop_date in event_date:
                done = True
            section_tag = soup.find('section', {'aria-labelledby': 'location-heading'})
            event_location = section_tag.find('p').get_text()
            event_section = soup.find('div', class_='has-user-generated-content')
            event_details = ''
            for p_tag in event_section.find_all('p'):
                event_details += p_tag.get_text()
            
            eventbrite['venue'] = venue
            eventbrite['event'] = event_name
            eventbrite['date'] = event_date
            eventbrite['age'] = np.nan
            eventbrite['price'] = 'Free'
            eventbrite['location'] = event_location
            eventbrite['event_description'] = event_details.strip()
            sf_events = sf_events.append(eventbrite, ignore_index=True)

KeyboardInterrupt: 

In [37]:
sf_events

,venue,event,date,age,price,location,event_description
0,"Celo • San Francisco, CA",DwellFi PropTech and Web3 Event,"Thu, February 23, 2023, 5:30 PM – 8:30 PM PST",NaN,The first time I ever saw a UFO was when I was...,"Celo 500 Treat Avenue San Francisco, CA 94110",T﻿his event is focused on PropTech and Web3.Pa...
1,"Anna Head Alumnae Hall • Berkeley, CA",YOUTH IMPACT SUMMIT - EMPOWERING YOUTH TO DRIV...,"Sat, February 25, 2023, 10:00 AM – 8:00 PM PST",NaN,The first time I ever saw a UFO was when I was...,"Anna Head Alumnae Hall 2537 Haste St, Berkeley...",This conference is targeted towards Nepali stu...
2,Hawthorn SF Nightclub and Lounge • San Francis...,YPOSF February Networking Happy Hour!,"Wed, February 22, 2023, 6:00 PM – 9:00 PM PST",NaN,The first time I ever saw a UFO was when I was...,Hawthorn SF Nightclub and Lounge 46 Geary Stre...,P﻿lease join us on February 22 for our marquis...
3,"Beacon Grand, A Union Square Hotel • San Franc...",San Francisco: Luxury Meetings Luncheon & Show...,"Thu, February 23, 2023, 11:00 AM – 2:00 PM PST",NaN,The first time I ever saw a UFO was when I was...,"Beacon Grand, A Union Square Hotel 450 Powell ...",Luxury Meetings has been connecting hoteliers ...
4,"Zeitgeist • San Francisco, CA",Bay Area Terra.do meetup,"Wed, February 22, 2023, 7:00 PM – 10:00 PM PST",NaN,The first time I ever saw a UFO was when I was...,"Zeitgeist 199 Valencia Street San Francisco, C...","Terra.do is a climate education, networking an..."
...,...,...,...,...,...,...,...
153,Evergreen Valley College | Cedro 102 (Classroo...,Evergreen Valley College - Time Management Wor...,"Multiple Dates Mon, Mar 6, 2023, 3:00 PM PST",NaN,The first time I ever saw a UFO was when I was...,Evergreen Valley College | Cedro 102 (Classroo...,"Being able to prioritize tasks, carve out time..."
154,"San Jose • San Jose, CA",San Jose CA-Intro to Real Estate Investing and...,"Multiple Dates Thu, Mar 16, 2023, 6:30 PM PST",NaN,The first time I ever saw a UFO was when I was...,San Jose Meeting link sent upon registration S...,Our Investor Community Intro meetings are a gr...
155,"Santa Rosa • Santa Rosa, CA",Conquer Your Fear of Public Speaking -Santa Ro...,"Multiple Dates Tue, Jun 18, 2024, 4:30 PM PDT",NaN,The first time I ever saw a UFO was when I was...,"Santa Rosa Santa Rosa Santa Rosa, CA 95401",Overcome public speaking fear with gradual exp...
156,"Santa Rosa • Santa Rosa, CA",How The Top 1% Of Leaders Manage Their Time Li...,"Multiple Dates Wed, Jan 24, 2024, 4:00 PM CDT",NaN,The first time I ever saw a UFO was when I was...,"Santa Rosa Online via Zoom Santa Rosa, CA 95404","In this workshop, participants will learn to:I..."


In [38]:
from opencage.geocoder import OpenCageGeocode
from shapely.geometry import Point
import pandas as pd
import numpy as np


key = '21b734acde524209895d1c01fc8e6eea'
geocoder = OpenCageGeocode(key)
locations = []

for index,row in sf_events.iterrows():
    v = row['location']
    results = geocoder.geocode(v)
    try:
        loc = Point(results[0]['geometry']['lat'],results[0]['geometry']['lng'])
        locations.append(loc)
    except:
        locations.append(np.nan)

sf_events['geolocation'] = locations

/Users/gauravmohan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


## Get Embeddings

In [40]:
import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding
import os
import openai

openai.apikey = os.getenv("OPENAI_API_KEY")
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 5000  # the maximum for text-embedding-ada-002 is 8191

In [42]:
def find_venue_type(venue):
    prompt = f'Provide a short description of the following venue in SF? Utilize the venues website or online search to find the answer.' + '\n' + 'Venue: ' + venue
    return prompt

def find_neighborhood(venue):
    prompt = "Given the following venue, provide the SF neighborhood that this venue is located in." + '\n' + 'Venue: ' + venue
    return prompt

In [44]:
venue_description = {}
for index, row in sf_events.iterrows():
    if row['venue'] not in venue_description:
        response = openai.Completion.create(
            model="text-curie-001",
            prompt=find_venue_type(row['venue']),
            temperature=0.3,
            max_tokens = 100,
        )
        response_text = response['choices'][0]['text'].strip()
        venue_description[row['venue']] = response_text
        sf_events.loc[index, 'venue_description'] = response_text

    else:
        sf_events.loc[index, 'venue_description'] = venue_description[row['venue']]  

In [45]:
venue_description = {}
for index, row in sf_events.iterrows():
    if row['venue'] not in venue_description:
        response = openai.Completion.create(
            model="text-curie-001",
            prompt=find_neighborhood(row['venue']),
            temperature=0.0,
            max_tokens = 50,
        )
        response_text = response['choices'][0]['text'].strip()
        venue_description[row['venue']] = response_text
        sf_events.loc[index, 'neighborhood'] = response_text

    else:
        sf_events.loc[index, 'neighborhood'] = venue_description[row['venue']]  

In [49]:
encoding = tiktoken.get_encoding(embedding_encoding)
eb_df = sf_events.applymap(str)
sf_events["combined"] = (
    "Venue: " + eb_df.venue.str.strip() + "; Venue Description: " + eb_df.venue_description.str.strip() + 
    "; Event: " + eb_df.event.str.strip() +  "; Event Description: " + eb_df.event_description.str.strip() +
    "; Neighborhood: " + eb_df.neighborhood.str.strip()
)

sf_events["n_tokens"] = sf_events.combined.apply(lambda x: len(encoding.encode(x)))

In [50]:
sf_events["embedding"] = sf_events.combined.apply(lambda x: get_embedding(x, engine=embedding_model))
sf_events.to_csv("eventbrite_with_embeddings.csv")